<a href="https://colab.research.google.com/github/barauna-lo/Cross_Correlation/blob/main/test_hr4_real.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

It's nice to comment the what all cells is doing! That will help us in the future to re-understand the code.

In [ ]:
#I put a # in front of the url for files that I don't want now
#This cell take the files from the HR4 site
!wget -i /content/teste_colab.txt \
  -P /content/files_download/

/content/teste_colab.txt: No such file or directory
No URLs found in /content/teste_colab.txt.


In [ ]:
#from google.colab import drive
#drive.mount('MyDrive')

Mounted at MyDrive


In [ ]:
#There's a 'ghost' file in the folder you need to remove
rmdir /content/files_download/.ipynb_checkpoints 

In [ ]:
!pip install healpy

import numpy as np
import h5py                              ## For to work with huge amounts of numerical data 
from scipy.optimize import curve_fit
import os                                ## Directory reading
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d   ## For to interpolate values between arrays?
import healpy as hp
from astropy.cosmology import FlatLambdaCDM, z_at_value ## Cosmological tools of astropy
import astropy.units as u


cosmo = FlatLambdaCDM(Om0=0.26,H0=72)

frequency = np.linspace(980.,1260.,31)   ## Frequency range of the BINGO 
redshift = 1420.40575177/frequency - 1.  ## Redshift compare with the HI frequency
distance = cosmo.comoving_distance(redshift).value * cosmo.h ## The distance at a given redshift, without unitis,

total = np.zeros(33,dtype=int)
j = 1
for i in os.listdir("/content/files_download/"):
    f = h5py.File("/content/files_download/"+i,"r")
    n = f["mass"].len()
    total[j] = total[j-1]+n
    j += 1

x = []#np.zeros(total[-1])
y = []#np.zeros(total[-1])
z = []#np.zeros(total[-1])
m = []#np.zeros(total[-1])
j = 1
for i in os.listdir("/content/files_download"):
	f = h5py.File("/content/files_download/"+i,"r")
	x[total[j-1]:total[j]] = np.asarray(f["x"])
	y[total[j-1]:total[j]] = np.asarray(f["y"])
	z[total[j-1]:total[j]] = np.asarray(f["z"])
	m[total[j-1]:total[j]] = np.asarray(f["mass"])
	j += 1

print ("data loading done")

at_value_z = np.arange(0.01,1.00,0.01)                   ## Defining the redshift range
at_value_r = cosmo.comoving_distance(at_value_z).value   ## Calculating the distance, without units
interp_rtoz = interp1d(at_value_r,at_value_z,fill_value="extrapolate")

r = []
for i in range(0,len(x)):
  r.append(np.sqrt(x[i]**2+y[i]**2+z[i]**2))

rx=[]
ry=[]
rz=[]
zeros = np.zeros(len(x))
k=0
def add_terms(a,b,c,x,y,z,l):
	a.append(x[l])
	b.append(y[l])
	c.append(z[l])
	return a,b,c
while(k<2):
	for i in range(0,len(x)):
		if r[i]<distance[k] and r[i]>distance[k+1]:
			add_terms(rx,ry,rz,x,y,z,i)
		else:
			add_terms(rx,ry,rz,zeros,zeros,zeros,i)			 
	# vec2pix function translate the vector x,y,z into the location in healpix pixel
	rpix = hp.vec2pix(256,rx,ry,rz)
	npix = hp.pixelfunc.nside2npix(256)
	# the pixel location histogram calculate the galaxy number in each pixel, divided by the volume in each pixel, which is the pixel area times the depth  
	rmap = np.histogram(rpix,bins=np.arange(0,npix+1,1))[0]
	volume = cosmo.comoving_volume(redshift[k]) - cosmo.comoving_volume(redshift[k+1]) ## Deep of pixel in redshift
	volume = volume.value * cosmo.h**3./hp.nside2npix(256)                             ## Divided by area?
	rrho = rmap/volume                                                                 ## Density 
# save the data in numpy binary format in the proper location
	np.save("/content/mocks/"+str(k)+"Mhz.npy",rrho)	
	k += 1

data loading done


/usr/local/lib/python3.7/dist-packages/healpy/pixelfunc.py:656: RuntimeWarning: divide by zero encountered in _vec2pix_ring
  return pixlib._vec2pix_ring(nside, x, y, z)
/usr/local/lib/python3.7/dist-packages/healpy/pixelfunc.py:656: RuntimeWarning: invalid value encountered in _vec2pix_ring
  return pixlib._vec2pix_ring(nside, x, y, z)


In [ ]:
#This cell do a 3D box with the points x,y,z, only to visualize the data.
#I've only tested it with only one file HR4.
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = Axes3D(fig)
ax.scatter(x,y,z,s=0.000001)
plt.show()